**Set environment**

In [1]:
suppressMessages(suppressWarnings(source("../config/config_sing.R")))
show_env()

You are in Singularity: singularity_proj_encode_fcc 
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei 
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/out 
CODE DIRECTORY (FD_CODE): /data/reddylab/Kuei/code 
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC 
PATH OF RESULTS (FD_RES): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc 
PATH OF LOG     (FD_LOG): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log 


**Check data**

In [2]:
fdiry = file.path(
    FD_RES, 
    "results", 
    "region", 
    "KS91_K562_ASTARRseq_peak_macs_input", 
    "annotation_tss_pol2")
dir(fdiry)

[1] "peak.annotation.K562.TSS.selected_by_highest_Pol2_signal.bed.gz"

In [4]:
fdiry = file.path(
    FD_RES, 
    "results", 
    "region", 
    "annotation_tss_pol2")
dir(fdiry)

[1] "description.tsv"                             
[2] "K562.TSS.selected_by_highest_Pol2_signal.bed"

## Import data

In [3]:
fdiry = file.path(
    FD_RES, 
    "results", 
    "region", 
    "annotation_tss_pol2")
fname = "description.tsv"
fpath = file.path(fdiry, fname)

dat = read_tsv(fpath)

dat_cnames = dat
print(dim(dat))
dat

Rows: 5 Columns: 2
── Column specification ─────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr (2): Name, Description

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] 5 2


Name,Description
<chr>,<chr>
Chrom,Chromosome
Start,TSS position
End,TSS position
Name,Gene name
Score,"highest level of pol2 chip seq (ENCFF914WIS.bigWig) at [TSS-500, TSS+500] among TSS isoform"


In [8]:
### init
annotation = "TSS_POL2"
label      = "TSS"
cnames = dat_cnames$Name
cnames = c("Chrom_ATAC", "Start_ATAC", "End_ATAC", cnames, "Overlap")

###
fdiry = file.path(
    FD_RES, 
    "results", 
    "region", 
    "KS91_K562_ASTARRseq_peak_macs_input", 
    "annotation_tss_pol2")
fname = "peak.annotation.K562.TSS.selected_by_highest_Pol2_signal.bed.gz"
fpath = file.path(fdiry, fname)

### read data
dat = read_tsv(fpath, col_names = cnames, show_col_types = FALSE)
dat = dat %>% dplyr::mutate(
    Peak_ATAC  = paste(Chrom_ATAC, Start_ATAC, End_ATAC, sep="_"),
    Annotation = annotation,
    Label      = label
)

### assign and show
dat_peak_annot = dat
print(dim(dat))
head(dat)

[1] 17751    12


Chrom_ATAC,Start_ATAC,End_ATAC,Chrom,Start,End,Name,Score,Overlap,Peak_ATAC,Annotation,Label
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>
chr1,17237,17772,chr1,17436,17437,MIR6859-1,9.43812,1,chr1_17237_17772,TSS_POL2,TSS
chr1,17237,17772,chr1,17436,17437,MIR6859-2,9.43812,1,chr1_17237_17772,TSS_POL2,TSS
chr1,17237,17772,chr1,17436,17437,MIR6859-3,9.43812,1,chr1_17237_17772,TSS_POL2,TSS
chr1,17237,17772,chr1,17436,17437,MIR6859-4,9.43812,1,chr1_17237_17772,TSS_POL2,TSS
chr1,28903,29613,chr1,29370,29371,WASH7P,0.00023,1,chr1_28903_29613,TSS_POL2,TSS
chr1,777949,779437,chr1,778634,778635,LOC100288069,230.56100,1,chr1_777949_779437,TSS_POL2,TSS


## Arrange and summarize

In [15]:
dat = dat_peak_annot
dat = dat %>% 
    dplyr::group_by(Peak_ATAC, Annotation, Label) %>%
    dplyr::summarise(Count = n(), .groups = "drop")

dat = dat %>% dplyr::rename("Peak" = "Peak_ATAC")

### assign and show
dat_peak_annot_count = dat
print(dim(dat))
head(dat)

[1] 15191     4


Peak,Annotation,Label,Count
<chr>,<chr>,<chr>,<int>
chr10_100009096_100010466,TSS_POL2,TSS,1
chr10_100185017_100187275,TSS_POL2,TSS,1
chr10_100228452_100230090,TSS_POL2,TSS,1
chr10_100267066_100268374,TSS_POL2,TSS,1
chr10_100285974_100287341,TSS_POL2,TSS,1
chr10_100346328_100347693,TSS_POL2,TSS,1


In [16]:
dat = dat_peak_annot_count
table(dat$Count)


    1     2     3     4     5     6    10 
12784  2283   107    12     2     2     1 

In [19]:
dat = dat_peak_annot_count
dat = dat %>% 
    tidyr::separate(Peak, c("Chrom", "Start", "End"), sep = "_")

Warning message:
“Expected 3 pieces. Additional pieces discarded in 9 rows [2933, 7882, 7883, 7884, 8863,
8864, 14715, 14716, 14717].”


In [21]:
dat = dat_peak_annot_count
dat[8864,]

Peak,Annotation,Label,Count
<chr>,<chr>,<chr>,<int>
chr22_KI270733v1_random_129881_130367,TSS_POL2,TSS,4


In [20]:
dat[8864,]

Chrom,Start,End,Annotation,Label,Count
<chr>,<chr>,<chr>,<chr>,<chr>,<int>
chr22,KI270733v1,random,TSS_POL2,TSS,4


In [13]:
50090348-50092805

[1] -2457

### Save results

In [22]:
dat_peak_annot_count %>% head

Peak,Annotation,Label,Count
<chr>,<chr>,<chr>,<int>
chr10_100009096_100010466,TSS_POL2,TSS,1
chr10_100185017_100187275,TSS_POL2,TSS,1
chr10_100228452_100230090,TSS_POL2,TSS,1
chr10_100267066_100268374,TSS_POL2,TSS,1
chr10_100285974_100287341,TSS_POL2,TSS,1
chr10_100346328_100347693,TSS_POL2,TSS,1


In [23]:
fdiry = file.path(FD_RES, "results", "region", "KS91_K562_ASTARRseq_peak_macs_input", "summary")
fname = "peak.summary.tss_pol2.tsv"
fpath = file.path(fdiry, fname)

dat = dat_peak_annot_count
write_tsv(dat, fpath)